# Imports

In [5]:
import os
import glob
import shutil

# Setup

In [6]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# !apt-get install lz4

In [8]:
# %%capture
# !pip install --upgrade duckdb papermill s5cmd dcm2niix pyplastimatch pyradiomics \
# highdicom pydicom seg-metrics idc-index itk SimpleITK pyaml

In [9]:
# %%capture
# from pyplastimatch.utils.install import install_precompiled_binaries
# install_precompiled_binaries()

In [ ]:
# dcmqi_release_url = "https://github.com/QIICR/dcmqi/releases/download/v1.3.4/dcmqi-1.3.4-linux.tar.gz"
# dcmqi_download_path = "dcmqi-1.3.4-linux.tar.gz"
# dcmqi_path = "dcmqi-1.3.4-linux"
# !wget -O $dcmqi_download_path $dcmqi_release_url\
# && tar -xvf $dcmqi_download_path\
# && mv $dcmqi_path/bin/* /bin\
# && rm -r $dcmqi_download_path $dcmqi_path

# Global Variables


In [17]:
#absolute path output
OUTPUT_PATH = ""#
#input MHUB zip files
mhubCompressedOutputFiles = []
#input rads ai compressed zip files
radsAiCompressedOutputFiles = []
#list of mhub_models
mhub_model_name_lst = []

# Paths definition

In [18]:
TASK_OUT_UNZIP = os.path.join(OUTPUT_PATH, "tasks_unzip")
#mhub_outputs
mhub_seg_out_unzip = os.path.join(TASK_OUT_UNZIP, "mhub_seg")
mhub_sr_out_unzip = os.path.join(TASK_OUT_UNZIP, "mhub_sr")
#final folder path
base_out_folder = os.path.join(OUTPUT_PATH, "aggregated_results")
##AI output folders
ai_seg_out_folder = os.path.join(base_out_folder, "ai_results", "dicom_seg")
ai_sr_out_folder = os.path.join(base_out_folder, "ai_results", "dicom_sr")
##papermill notebooks log outputs
papermill_logs_out_folder = os.path.join(base_out_folder, "papermill_logs")

In [19]:
def setup_folders():
  """
  creates environment data and results folder, first deletes current ones and then creates new ones
  """
  for folder_setup in [TASK_OUT_UNZIP,
                       mhub_seg_out_unzip,
                       mhub_sr_out_unzip,
                       base_out_folder,
                       ai_seg_out_folder,
                       ai_sr_out_folder,
                       papermill_logs_out_folder]:
    !rm -rf $folder_setup
    !mkdir -p $folder_setup

In [20]:
setup_folders()

# Create model-specific output folders

In [21]:
for model_name in mhub_model_name_lst:
  #create models unzip paths
  for path_model_create in [mhub_seg_out_unzip,
                            mhub_sr_out_unzip]:
    temp_path = os.path.join(path_model_create, model_name)
    !mkdir -p $temp_path
  ##create models out paths
  for path_model_create in [ai_seg_out_folder, ai_sr_out_folder,
                            papermill_logs_out_folder]:
    temp_path = os.path.join(path_model_create, model_name)
    !mkdir -p $temp_path

# Unzip all outputs and aggregate into final folder

mhub ai segs -- Unzip and move to output directory

In [ ]:
for idx_model, model_name in enumerate(mhub_model_name_lst):
  #download archive
  archive_temp = mhubCompressedOutputFiles[idx_model]
  archive_filemame = archive_temp.split("/")[-1]
  zip_ddl_path = os.path.join(mhub_seg_out_unzip, model_name)
  # !gsutil -m cp -r $archive_temp $zip_ddl_path
  !cp $archive_temp $zip_ddl_path
  zip_path = os.path.join(mhub_seg_out_unzip, model_name, archive_filemame)
  !lz4 -dc < $zip_path | tar xvf - -C $zip_ddl_path
  #move archive contents to output folder
  dcm_files_lst = glob.glob(os.path.join(zip_ddl_path, "**", "*.dcm"), recursive=True)
  ##get image_serieUID
  ##define output path for each DCM file, add to the .dcm  a prefix image_serieUID
  for in_dicom_file_path in dcm_files_lst:
    image_segmented_serieUID = in_dicom_file_path.split("/")[-2].strip()
    ##make sure output_path_folder is defined
    out_folder_path = os.path.join(ai_seg_out_folder, model_name,
                 image_segmented_serieUID)
    !mkdir -p $out_folder_path
    out_dcm_file_path = os.path.join(out_folder_path,
                                     image_segmented_serieUID+"_"+in_dicom_file_path.split("/")[-1].strip())
    #copy to output folder
    !cp $in_dicom_file_path $out_dcm_file_path

Unzip mhub ai srs

In [ ]:
for idx_model, model_name in enumerate(mhub_model_name_lst):
  archive_temp = radsAiCompressedOutputFiles[idx_model]
  archive_filemame = archive_temp.split("/")[-1]
  zip_ddl_path = os.path.join(mhub_sr_out_unzip, model_name)
  # !gsutil -m cp -r $archive_temp $zip_ddl_path
  !cp $archive_temp $zip_ddl_path
  zip_path = os.path.join(mhub_sr_out_unzip, model_name, archive_filemame)
  !lz4 -dc < $zip_path | tar xvf - -C $zip_ddl_path
  #move archive contents to output folder
  dcm_files_lst = glob.glob(os.path.join(zip_ddl_path, "**", "*.dcm"), recursive=True)
  ##get image_serieUID
  ##define output path for each DCM file, add to the .dcm  a prefix image_serieUID
  for in_dicom_file_path in dcm_files_lst:
    image_segmented_serieUID = in_dicom_file_path.split("/")[-2].strip()
    ##make sure output_path_folder is defined
    out_folder_path = os.path.join(ai_sr_out_folder, model_name,
                 image_segmented_serieUID)
    !mkdir -p $out_folder_path
    out_dcm_file_path = os.path.join(out_folder_path,
                                     image_segmented_serieUID+"_"+in_dicom_file_path.split("/")[-1].strip())
    #copy to output folder
    !cp $in_dicom_file_path $out_dcm_file_path

Move papermill logs to archive

In [ ]:
for idx_model, model_name in enumerate(mhub_model_name_lst):
  #get papermill logs for ai_seg combine,ai_sr,eval_quant tasks
  papermill_combine_ai_seg_path = glob.glob(os.path.join(mhub_seg_out_unzip, model_name, "**", "*.ipynb"), recursive=True)[0]
  papermil_ai_sr_path = glob.glob(os.path.join(mhub_sr_out_unzip, model_name, "**", "*.ipynb"), recursive=True)[0]
  ##copy MHUB outputs and eval to archive folder
  out_papermil_ai_folder = os.path.join(papermill_logs_out_folder, model_name)
  !mkdir -p $out_papermil_ai_folder
  #copy to output folder
  ##define out paths
  out_papermil_ai_sr_file_path = os.path.join(out_papermil_ai_folder, papermil_ai_sr_path.split("/")[-1])
  out_papermil_ai_seg_file_path = os.path.join(out_papermil_ai_folder, papermill_combine_ai_seg_path.split("/")[-1])
  !cp $papermil_ai_sr_path $out_papermil_ai_sr_file_path
  !cp $papermill_combine_ai_seg_path $out_papermil_ai_seg_file_path

# Create LZ4 archive

In [ ]:
output_lz4_archive = os.path.join(OUTPUT_PATH, "agg_archive.tar.lz4")

In [ ]:
# # Compress output data and move it to Cromwell root directory
!tar -C $OUTPUT_PATH -cvf - aggregated_results | lz4 > $output_lz4_archive